In [1]:
from enum import IntEnum
import time
import jsonrpclib
import subprocess
from subprocess import PIPE, Popen
from threading  import Thread
import sys
import re

import PySimpleGUI as sg

from gym import Env, error, spaces, utils
from stable_baselines3 import DQN, PPO, A2C, TD3, SAC
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.env_checker import check_env
import numpy as np

import os
import requests
import shutil
import tempfile
import xml.etree.ElementTree as ET
from io import StringIO, BytesIO

import cv2
import numpy as np
import torch
from PIL import Image

import olympe
from olympe.messages.ardrone3.Piloting import TakeOff, Landing, moveBy, PCMD, moveTo
from olympe.messages.ardrone3.PilotingState import FlyingStateChanged, GpsLocationChanged, moveToChanged
from olympe.enums.ardrone3.PilotingState import FlyingStateChanged_State as FlyingState
from olympe.messages.ardrone3.GPSSettingsState import GPSFixStateChanged
from olympe.messages.gimbal import set_target
from olympe.messages.camera import (
    set_camera_mode,
    set_photo_mode,
    take_photo,
    photo_progress,
)

from pynput.keyboard import Listener, Key, KeyCode
from collections import defaultdict

olympe.log.update_config({
    "loggers": {
        "olympe": {
                "handlers": []
            }
        },
        "ulog": {
            "level": "OFF",
            "handlers": [],
        }
})

In [ ]:
import PySimpleGUI as sg
import time

def start_gui_2():
    layout = [
        [
            sg.Column([
                [sg.Text("Reward", size=(20, 2), justification='center', font=("Helvetica, 14"))],
                [sg.Text("Reward", size=(20, 3), key="-REWARD-", justification='center')],
                [
                    sg.Button("Start", key="START", size=(10, 2), mouseover_colors = (None, None),), 
                    sg.Button("Terminate", key="TERMINATE", size=(10, 2)),
                ]
            ],
            element_justification='center'),
        ],
    ]

    window = sg.Window("Results", layout, margins=(25, 25))

    timer_running = True
    
    # Create an event loop
    while True:
        event, values = window.read(timeout=10)

        if event == "TERMINATE" or event == sg.WIN_CLOSED:
            break
        elif event == "START":
            timer_running = not timer_running
        if timer_running:
            window['-REWARD-'].update(time.strftime("%H:%M:%S"))
            time.sleep(3)
            
    window.close()

In [ ]:
def init_gui(window, event, values):
    # Create an event loop
    while True:
        event, values = window.read(timeout=10)
        
        if event == "TERMINATE" or event == sg.WIN_CLOSED:
            break
            
def start_gui_3():
    layout = [
        [
            sg.Column([
                [sg.Text("Reward", size=(20, 2), justification='center', font=("Helvetica, 14"))],
                [sg.Text("Reward", size=(20, 3), key="-REWARD-", justification='center')],
                [
                    sg.Button("Start", key="START", size=(10, 2), mouseover_colors = (None, None),), 
                    sg.Button("Terminate", key="TERMINATE", size=(10, 2)),
                ]
            ],
            element_justification='center'),
        ],
    ]

    window = sg.Window("Results", layout, margins=(25, 25))

    timer_running = False
    event, values = None, None
    gui = Thread(target=init_gui, args=[window, event, values], daemon=True)
    gui.start()
    while True:
        if event == "START":
            timer_running = not timer_running
        if timer_running:
            window['-REWARD-'].update(time.strftime("%H:%M:%S"))
            time.sleep(1)
        if not gui.is_alive():
            break
    
    window.close()

In [ ]:
start_gui_3()

In [ ]:
import PySimpleGUI as sg
import time
import threading

ending = False
def show_time(window):
    while True:
        window['-REWARD-'].update(time.strftime("%H:%M:%S"))
        time.sleep(1)
        
        if ending:
            break

def start_gui():
    layout = [
        [
            sg.Column([
                [sg.Text("Reward", size=(20, 2), justification='center', font=("Helvetica, 14"))],
                [sg.Text("Reward", size=(20, 3), key="-REWARD-", justification='center')],
                [
                    sg.Button("Start", key="START", size=(10, 2)), 
                    sg.Button("Terminate", key="TERMINATE", size=(10, 2)),
                ]
            ],
            element_justification='center'),
        ],
    ]

    window = sg.Window("Results", layout, margins=(25, 25))

    # Create an event loop
    while True:
        event, values = window.read()

        if event == "TERMINATE" or event == sg.WIN_CLOSED:
            ending = True
            break
        elif event == "START":
            threading.Thread(target=show_time, args=[window], daemon=True).start()

    window.close()

In [ ]:
import PySimpleGUI as sg

sg.theme('DarkBrown1')

layout = [  [sg.Text('Stopwatch', size=(20, 2), justification='center')],
            [sg.Text(size=(10, 2), font=('Helvetica', 20), justification='center', key='-OUTPUT-')],
            [sg.T(' ' * 5), sg.Button('Start/Stop', focus=True), sg.Quit()]]

window = sg.Window('Stopwatch Timer', layout)

timer_running, counter = True, 0

while True:                                 # Event Loop
    event, values = window.read(timeout=10) # Please try and use as high of a timeout value as you can
    if event in (sg.WIN_CLOSED, 'Quit'):             # if user closed the window using X or clicked Quit button
        break
    elif event == 'Start/Stop':
        timer_running = not timer_running
    if timer_running:
        window['-OUTPUT-'].update('{:02d}:{:02d}.{:02d}'.format((counter // 100) // 60, (counter // 100) % 60, counter % 100))
        counter += 1
window.close()

In [ ]:
#!/usr/bin/python3
import threading
import time
import PySimpleGUI as sg


"""
    DESIGN PATTERN - Multithreaded Long Tasks GUI
    Presents one method for running long-running operations in a PySimpleGUI environment.
    The PySimpleGUI code, and thus the underlying GUI framework, runs as the primary, main thread
    The "long work" is contained in the thread that is being started.

    July 2020 - Note that this program has been updated to use the new Window.write_event_value method.
    This method has not yet been ported to the other PySimpleGUI ports and is thus limited to the tkinter ports for now.
    
    Internally to PySimpleGUI, a queue.Queue is used by the threads to communicate with main GUI code
    The PySimpleGUI code is structured just like a typical PySimpleGUI program.  A layout defined,
        a Window is created, and an event loop is executed.


    This design pattern works for all of the flavors of PySimpleGUI including the Web and also repl.it
    You'll find a repl.it version here: https://repl.it/@PySimpleGUI/Async-With-Queue-Communicationspy
"""

# ending = False

def long_operation_thread(seconds, window):
    """
    A worker thread that communicates with the GUI through a queue
    This thread can block for as long as it wants and the GUI will not be affected
    :param seconds: (int) How long to sleep, the ultimate blocking call
    :param gui_queue: (queue.Queue) Queue to communicate back to GUI that task is completed
    :return:
    """
    while True:
        print(time.strftime("%H:%M:%S"))
        time.sleep(1)
#         if ending is True:
#             break
#     print('Starting thread - will sleep for {} seconds'.format(seconds))
#     time.sleep(seconds)                  # sleep for a while
#     window.write_event_value('-THREAD-', '** DONE **')  # put a message into queue for GUI


def the_gui():
    """
    Starts and executes the GUI
    Reads data from a Queue and displays the data to the window
    Returns when the user exits / closes the window
    """
    sg.theme('Light Brown 3')

    layout = [[sg.Text('Long task to perform example')],
              [sg.Output(size=(70, 12))],
              [sg.Text('Number of seconds your task will take'),
                  sg.Input(key='-SECONDS-', size=(5, 1)),
                  sg.Button('Do Long Task', bind_return_key=True)],
              [sg.Button('Click Me'), sg.Button('Exit')], ]

    window = sg.Window('Multithreaded Window', layout)

    # --------------------- EVENT LOOP ---------------------
    while True:
        event, values = window.read()
        if event in (sg.WIN_CLOSED, 'Exit'):
#             ending = True
            break
        elif event.startswith('Do'):
            seconds = int(values['-SECONDS-'])
            print('Thread ALIVE! Long work....sending value of {} seconds'.format(seconds))
            threading.Thread(target=long_operation_thread, args=(seconds, window,), daemon=True).start()
        elif event == 'Click Me':
            print('Your GUI is alive and well')
        elif event == '-THREAD-':
            print('Got a message back from the thread: ', values[event])

    # if user exits the window, then close the window and exit the GUI func
    window.close()
    return

if __name__ == '__main__':
    the_gui()
    print('Exiting Program')


In [ ]:
import PySimpleGUI as sg

# Define the window's contents
layout = [[sg.Text("What's your name?")],
          [sg.Input(key='-INPUT-')],
          [sg.Text(size=(60,1), key='-OUTPUT-')],
          [sg.Button('Ok'), sg.Button('Quit')]]

# Create the window
window = sg.Window('Window Title', layout)

# Display and interact with the Window using an Event Loop
while True:
    event, values = window.read()
    # See if user wants to quit or window was closed
    if event == sg.WINDOW_CLOSED or event == 'Quit':
        break
    # Output a message to the window
    window['-OUTPUT-'].update('Hello ' + values['-INPUT-'] + "! Thanks for trying PySimpleGUI " + time.strftime("%H:%M:%S"))

# Finish up by removing from the screen
window.close()

In [2]:
class KeyboardCtrl(Listener):
    def __init__(self, ctrl_keys=None):
        self._key_pressed = defaultdict(lambda: False)
        self._last_action_ts = defaultdict(lambda: 0.0)
        super().__init__(on_press=self._on_press)
        self.start()

    def _on_press(self, key):
        if isinstance(key, KeyCode):
            self._key_pressed[key.char] = True
        elif isinstance(key, Key):
            self._key_pressed[key] = True
        if self._key_pressed[Key.esc]:
            return False
        else:
            return True
        
    def quit(self):
        return not self.running or self._key_pressed[Key.esc]

In [3]:
Action = IntEnum(
    'ACTION',
#     'UP DOWN LEFT RIGHT FORWARD BACKWARD HOVER',
    'FORWARD BACKWARD LEFT RIGHT FORWARD_LEFT FORWARD_RIGHT BACKWARD_LEFT BACKWARD_RIGHT HOVER',
    start=0
)

class AnafiEnv(Env):
    def __init__(self, 
                 DRONE_IP="10.202.0.1", 
                 x_move_dist=1, 
                 y_move_dist=1.64,
                 boundaries={"x":5, "y":5, "z_min":3, "z_max":8}):
        
        super(AnafiEnv, self).__init__()
        
        self.x_move_dist = x_move_dist
        self.y_move_dist = y_move_dist
        self.boundaries = boundaries
        self.DRONE_IP = DRONE_IP
        
        self.model = torch.hub.load('../yolov5', 'custom', path='../weights/best.pt', source='local', verbose=False)
        self.model.conf = 0.7
        self.model.classes = [0]
        
        self.sphinx = jsonrpclib.Server('http://127.0.0.1:8383')
        self.sphinx.SetParam(machine='anafi4k', object='lipobattery/lipobattery', parameter='discharge_speed_factor', value='0')
        
        self.drone = olympe.Drone(self.DRONE_IP)
        self.drone.connect()
        self.setup_photo_single_mode()
        self.takeoff()
        self.home = self.drone.get_state(GpsLocationChanged)
        self.agent_coord = np.zeros(3)
        
        self.action_space = spaces.Discrete(len(Action))
        self.observation_space = spaces.Box(low=-1, high=1, shape=(3,), dtype=np.float32)
        
        ON_POSIX = 'posix' in sys.builtin_module_names
        cmd = "parrot-gz topic -e /gazebo/default/pose/info | grep -A 5 'name: \"anafi4k\"'"
        p = Popen(cmd, stdout=PIPE, bufsize=1, close_fds=ON_POSIX, shell=True)
        self.odom_thread = Thread(target=self.set_current_agent_coord, args=[p.stdout], daemon=True)
        self.odom_thread.start()
        
        self.step_count = 0
        self.rewards = []
        
        self.drone(PCMD(1, 0, 0, 0, 35, 0))
        time.sleep(0.05)
#         while self.agent_coord[2] < boundaries["z_min"]:
# #             gaz = max(10, 100 * 3**( (boundaries["z_min"]-self.agent_coord[2]) - boundaries["z_min"] ))
# #             print(2**( (boundaries["z_min"]-self.agent_coord[2]) - boundaries["z_min"] ))
#             gaz = max(10, 100 * 3**( (boundaries["z_min"]-self.agent_coord[2]) - boundaries["z_min"] ))
#             print(int(gaz))
#             self.drone(PCMD(1, 0, 0, 0, int(gaz), 0))
#             time.sleep(0.05)
    
    def setup_photo_single_mode(self):
        self.drone(set_camera_mode(cam_id=0, value="photo")).wait()

        self.drone(
            set_photo_mode(
                cam_id=0,
                mode="single",
                format="rectilinear",
                file_format="jpeg",
                burst="burst_14_over_1s",
                bracketing="preset_1ev",
                capture_interval=0.0,
                )
            ).wait()
        
        self.drone(
            set_target(
                gimbal_id=0,
                control_mode="position",
                yaw_frame_of_reference="none",
                yaw=0.0,
                pitch_frame_of_reference="absolute",
                pitch=-90.0,
                roll_frame_of_reference="none",
                roll=0.0,
                )
            ).wait()
        
    def take_photo_single(self):

        # Drone web server URL
        ANAFI_URL = "http://{}/".format(self.DRONE_IP)

        # Drone media web API URL
        ANAFI_MEDIA_API_URL = ANAFI_URL + "api/v1/media/medias/"

        self.drone(PCMD(1, 0, 0, 0, 0, 0) >> FlyingStateChanged(state="hovering", _timeout=5)).wait()
        photo_saved = self.drone(photo_progress(result="photo_saved", _policy="wait"))
        self.drone(take_photo(cam_id=0)).wait()
        if not photo_saved.wait().success():
            print("Photos not saved")
        media_id = photo_saved.received_events().last().args["media_id"]

        # download the photos associated with this media id
        media_info_response = requests.get(ANAFI_MEDIA_API_URL + media_id)
        media_info_response.raise_for_status()
        for resource in media_info_response.json()["resources"]:
            image_response = requests.get(ANAFI_URL + resource["url"], stream=True)
            image_response.raise_for_status()
            img = Image.open(BytesIO(image_response.content))
            result = self.model(img)
#             result.print()
#             result.show()
    
        return result.xyxy[0].shape[0]
        
        # TODO: Delete the image file in the web server after processing to avoid accumulation
    
    def get_current_agent_coord(self):
        return self.agent_coord
    
    def set_current_agent_coord(self, output):
        for line in iter(output.readline, b''):
            line = str(line)
            
            if "x:" in line:
                x = re.findall(r"[-+]?\d*\.\d+|\d+", line)[0]
                self.agent_coord[0] = float(x)
            elif "y:" in line:
                y = re.findall(r"[-+]?\d*\.\d+|\d+", line)[0]
                self.agent_coord[1] = float(y)
            elif "z:" in line:
                z = re.findall(r"[-+]?\d*\.\d+|\d+", line)[0]
                self.agent_coord[2] = float(z)
            
    def step(self, action):
        
        done = False
        
        x, y, z = self.agent_coord
        
        # change invalid actions to hover
        if self.boundaries["x"] - abs(x) < self.x_move_dist:
            if x > 0 and (action == Action.FORWARD or action == Action.FORWARD_LEFT or action == Action.FORWARD_RIGHT):
                action = Action.HOVER
            elif x < 0 and (action == Action.BACKWARD or action == Action.BACKWARD_LEFT or action == Action.BACKWARD_RIGHT):
                action = Action.HOVER
        if self.boundaries["y"] - abs(y) < self.y_move_dist:
            if y > 0 and (action == Action.LEFT or action == Action.BACKWARD_LEFT or action == Action.FORWARD_LEFT):
                action = Action.HOVER
            elif y < 0 and (action == Action.RIGHT or action == Action.BACKWARD_RIGHT or action == Action.FORWARD_RIGHT):
                action = Action.HOVER
#         elif (z - self.boundaries["z_min"] < self.move_dist or z < self.boundaries["z_min"]) and action == Action.DOWN:
#             action = Action.HOVER
#         elif (self.boundaries["z_max"] - z < self.move_dist or z > self.boundaries["z_max"]) and action == Action.UP:
#             action = Action.HOVER
            
       
        self.take_action(action)
        
        self.step_count += 1
        
        obs = [0.5, 0.5, 0.5]
        reward = self.take_photo_single()
#         reward = 0
        self.rewards.append(reward)

#         obs = self.agent_coord / self.boundary
            
#         if (self.agent_coord >= self.).any() or self.step_count >= 10:
#             done = True
#             self.step_count = 0
        
        return obs, reward, done, {}

    def take_action(self, action):
        if action == Action.FORWARD:
            self.move(self.x_move_dist, 0, 0, 0)
        elif action == Action.BACKWARD:
            self.move(-self.x_move_dist, 0, 0, 0)
        elif action == Action.RIGHT:
            self.move(0, self.y_move_dist, 0, 0)
        elif action == Action.LEFT:
            self.move(0, -self.y_move_dist, 0, 0)
        elif action == Action.FORWARD_RIGHT:
            self.move(self.x_move_dist, self.y_move_dist, 0, 0)
        elif action == Action.FORWARD_LEFT:
            self.move(self.x_move_dist, -self.y_move_dist, 0, 0)
        elif action == Action.FORWARD_RIGHT:
            self.move(-self.x_move_dist, self.y_move_dist, 0, 0)
        elif action == Action.FORWARD_LEFT:
            self.move(-self.x_move_dist, -self.y_move_dist, 0, 0)
        elif action == Action.HOVER:
            self.move(0, 0, 0, 0)
    
    def move(self, lin_x, lin_y, lin_z, ang_yaw):
        success =  self.drone(
                            moveBy(lin_x, lin_y, -lin_z, ang_yaw)  # (forward-backward, right-left, down-up, clockwise-anticlockwise)
                            >> FlyingStateChanged(state="hovering", _timeout=5)
                            ).wait().success()
                            
    def takeoff(self):
        assert self.drone(
                    FlyingStateChanged(state="hovering", _policy="check")
                    | (
                        GPSFixStateChanged(fixed=1, _timeout=10)
                        >> (
                            TakeOff(_no_expect=True)
                            & FlyingStateChanged(state="hovering", _policy="wait", _timeout=5)
                        )
                    )
                ).wait().success()
        
    def reset(self):
        latitude, longitude, altitude = self.home["latitude"], self.home["longitude"], self.home["altitude"]
        
        resetAction = self.drone(
            moveTo(latitude,  longitude, altitude, "HEADING_DURING", 90.0)
            >> moveToChanged(status="DONE", _timeout=15)
        ).wait()
    
#         return self.agent_coord / self.boundary
        return [0.5, 0.5, 0.5]
    
    def render(self, mode='human'):
        pass
    
    def close(self):
        self.drone(PCMD(1, 0, 0, 0, 0, 0) >> FlyingStateChanged(state="hovering", _timeout=5)).wait()
        assert self.drone(Landing() >> FlyingStateChanged(state="landed")).wait().success()
        self.drone.disconnect()

In [ ]:
del env

In [5]:
layout = [
    [
            sg.Column([
                [sg.Text("Reward", size=(20, 2), justification='center', font=("Helvetica, 14"))],
                [sg.Text("0", size=(20, 3), key="-REWARD-", justification='center', font=("Helvetica", 12))],
            ],
            element_justification='center'),
        ],
    ]  

env = AnafiEnv(x_move_dist=1, y_move_dist=1.64, boundaries={"x":2, "y":3.28, "z_min":1.1, "z_max":70})
control = KeyboardCtrl()
window = sg.Window("Results", layout, margins=(25, 25))

while not control.quit():
    event, values = window.read(timeout=10)
    
    action = env.action_space.sample()
    state, reward, done, _ = env.step(action)
    
    window['-REWARD-'].update(str(reward))
    if event == sg.WIN_CLOSED:
        break

window.close()
env.reset()
env.close()

YOLOv5 🚀 v5.0-392-gb894e69 torch 1.9.0+cu111 CPU

Fusing layers... 
Model Summary: 308 layers, 21037638 parameters, 0 gradients
Adding AutoShape... 
Creating pomp loop
device callbacks have been added to arsdk_ctrl
Creating pomp timer
Creating pomp loop
Creating pomp timer
New device has been detected: 'ANAFI-0000000'
Connecting to device: ANAFI-0000000
Connection in progress...
Connected to device: ANAFI-0000000
{'json': {'c2d_port': 2233,
          'c2d_update_port': 51,
          'c2d_user_port': 21,
          'proto_v': 3,
          'qos_mode': 0,
          'status': 0}}
Creating pomp loop
Command interface has been created: itf=None
Creating pomp loop
rth.ending_behavior(ending_behavior=ending_behavior.hovering)
Creating pomp loop
common.Common.AllStates() has been sent to the device
common.CommonState.BatteryStateChanged(percent=100)
common.ARLibsVersionsState.DeviceLibARCommandsVersion(version='4.0.0.4')
common.CommonState.BootId(bootId='269457F4D16A66765289CA84FBE08F15')
ardron

camera.recording_mode(cam_id=0, mode=recording_mode.standard, resolution=resolution.res_uhd_4k, framerate=framerate.fps_30, hyperlapse=hyperlapse_value.ratio_15, bitrate=10000000, list_flags='')
camera.white_balance(cam_id=0, mode=white_balance_mode.automatic, temperature=white_balance_temperature.t_1500, lock=state.inactive, list_flags='')
camera.exposure_settings(cam_id=0, mode=exposure_mode.automatic, manual_shutter_speed=shutter_speed.shutter_1_over_10000, manual_shutter_speed_capabilities='shutter_1_over_10000|shutter_1_over_8000|shutter_1_over_6400|shutter_1_over_5000|shutter_1_over_4000|shutter_1_over_3200|shutter_1_over_2500|shutter_1_over_2000|shutter_1_over_1600|shutter_1_over_1250|shutter_1_over_1000|shutter_1_over_800|shutter_1_over_640|shutter_1_over_500|shutter_1_over_400|shutter_1_over_320|shutter_1_over_240|shutter_1_over_200|shutter_1_over_160|shutter_1_over_120|shutter_1_over_100|shutter_1_over_80|shutter_1_over_60|shutter_1_over_50|shutter_1_over_40|shutter_1_over_30

rth.auto_trigger_mode(mode=auto_trigger_mode.on)
rth.state(state=state.unavailable, reason=state_reason.disabled)
ardrone3.PilotingSettingsState.MotionDetection(enabled=0)
rth.takeoff_location(latitude=48.87889999476982, longitude=2.3677789215840357, altitude=2.0, fixed_before_takeoff=1)
rth.preferred_home_type(type=home_type.takeoff)
ardrone3.GPSSettingsState.HomeTypeChanged(type=HomeTypeChanged_Type.TAKEOFF)
common.SettingsState.ProductNameChanged(name='Parrot Drone')
wifi.supported_security_types(types='open|wpa2')
wifi.security_changed(type=security_type.open, key='', key_type=security_key_type.plain)
wifi.ap_channel_changed(type=selection_type.auto_all, band=-1, channel=255)
wifi.country_changed(selection_mode=country_selection.auto, code='FR')
wifi.environment_changed(environment=environment.outdoor)
gimbal.relative_attitude_bounds(gimbal_id=0, min_yaw=0.0, max_yaw=0.0, min_pitch=-135.0, max_pitch=105.0, min_roll=-38.0, max_roll=38.0, list_flags='')
gimbal.absolute_attitude_bound

camera.photo_mode(cam_id=0, mode=photo_mode.single, format=photo_format.rectilinear, file_format=photo_file_format.jpeg, burst=burst_value.burst_14_over_1s, bracketing=bracketing_preset.preset_1ev, capture_interval=0.0, list_flags='')
gimbal.set_target(0, <control_mode.position: 0>, <frame_of_reference.none: 0>, 0.0, <frame_of_reference.absolute: 2>, -90.0, <frame_of_reference.none: 0>, 0.0) has been sent to the device
ardrone3.Piloting.TakeOff() has been sent to the device
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.motor_ramping)
common.RunState.RunIdChanged(runId='F702B3')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.takingoff)
follow_me.mode_info(mode=mode.look_at, missing_requirements='drone_calibrated|drone_gps_good_accuracy|target_gps_good_accuracy|target_barometer_ok|drone_far_enough|drone_high_enough|target_good_speed|drone_close_enough', improvements='drone_calibrated|drone_gps_good_accuracy|target_gps_good_accuracy|tar

camera.photo_state(cam_id=0, available=availability.available, state=state.active, list_flags='')
camera.photo_progress(cam_id=0, result=photo_result.taking_photo, photo_count=0, media_id='', list_flags='')
media_event(name=media_created, media_id=10000011, photo_mode=SINGLE)
camera.photo_progress(cam_id=0, result=photo_result.photo_taken, photo_count=1, media_id='', list_flags='')
camera.photo_progress(cam_id=0, result=photo_result.photo_saved, photo_count=0, media_id='10000011', list_flags='')
camera.photo_state(cam_id=0, available=availability.available, state=state.inactive, list_flags='')
mediastore.counters(video_media_count=0, photo_media_count=11, video_resource_count=0, photo_resource_count=11)
ardrone3.Piloting.PCMD(1, 0, 0, 0, 0, 0) has been sent to the device
animation.availability(values='horizontal_180_photo_panorama|vertical_180_photo_panorama|spherical_photo_panorama')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.hovering)
animation.availabil

...

camera.photo_state(cam_id=0, available=availability.available, state=state.active, list_flags='')
camera.photo_progress(cam_id=0, result=photo_result.taking_photo, photo_count=0, media_id='', list_flags='')
camera.photo_progress(cam_id=0, result=photo_result.photo_taken, photo_count=1, media_id='', list_flags='')
camera.photo_progress(cam_id=0, result=photo_result.photo_saved, photo_count=0, media_id='10000092', list_flags='')
camera.photo_state(cam_id=0, available=availability.available, state=state.inactive, list_flags='')
mediastore.counters(video_media_count=0, photo_media_count=92, video_resource_count=0, photo_resource_count=92)
ardrone3.Piloting.PCMD(1, 0, 0, 0, 0, 0) has been sent to the device
animation.availability(values='horizontal_180_photo_panorama|vertical_180_photo_panorama|spherical_photo_panorama')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.hovering)
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reve

mediastore.counters(video_media_count=0, photo_media_count=96, video_resource_count=0, photo_resource_count=96)
ardrone3.Piloting.moveBy(0.0, 0.0, 0.0, 0.0) has been sent to the device
animation.availability(values='horizontal_180_photo_panorama|vertical_180_photo_panorama|spherical_photo_panorama')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.hovering)
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up|horizontal_180_photo_panorama|vertical_180_photo_panorama|spherical_photo_panorama')
ardrone3.PilotingEvent.moveByEnd(dX=0.0, dY=0.0, dZ=0.0, dPsi=0.0, error=MoveByEnd_Error.ok)
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.flying)
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up|horizontal_180_photo_panorama|vertical_180_photo_pan

animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.flying)
animation.availability(values='')
ardrone3.PilotingEvent.moveByEnd(dX=-0.00411086343228817, dY=-1.5406402349472046, dZ=7.56978988647461e-05, dPsi=-0.002243638038635254, error=MoveByEnd_Error.ok)
animation.availability(values='horizontal_180_photo_panorama|vertical_180_photo_panorama|spherical_photo_panorama')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.hovering)
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up|horizontal_180_photo_panorama|vertical_180_photo_panorama|spherical_photo_panorama')
ardrone3.Piloting.PCMD(1, 0, 0, 0, 0, 0) has been sent to the device
animation.availability(values='horizontal_panorama|dronie|horizontal_reveal|vertical_reveal|twist_up')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChang

camera.photo_progress(cam_id=0, result=photo_result.photo_taken, photo_count=1, media_id='', list_flags='')
camera.photo_progress(cam_id=0, result=photo_result.photo_saved, photo_count=0, media_id='10000104', list_flags='')
camera.photo_state(cam_id=0, available=availability.available, state=state.inactive, list_flags='')
mediastore.counters(video_media_count=0, photo_media_count=104, video_resource_count=0, photo_resource_count=104)
ardrone3.Piloting.moveTo(48.8788998249266, 2.3677785277039565, 0.7650997638702393, <MoveTo_Orientation_mode.HEADING_DURING: 3>, 90.0) has been sent to the device
animation.availability(values='horizontal_180_photo_panorama|vertical_180_photo_panorama|spherical_photo_panorama')
ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.hovering)
ardrone3.PilotingState.moveToChanged(latitude=48.8788998249266, longitude=2.3677785277039565, altitude=0.7650997638702393, orientation_mode=MoveToChanged_Orientation_mode.HEADING_DURING, heading=90.0, 

ardrone3.PilotingState.FlyingStateChanged(state=FlyingStateChanged_State.landed)
we are not disconnected yet
Disconnected from device: ANAFI-0000000
disconnected from device: b'10.202.0.1'
Disconnection with the device OK. IP: b'10.202.0.1'
olympe.media shutdown
Pomp loop has been destroyed
Pomp loop has been destroyed
pdraw callbacks thread loop stopped


In [ ]:
env.rewards

In [ ]:
layout = [
        [
            sg.Column([
                [sg.Text("Reward", size=(20, 2), justification='center', font=("Helvetica, 14"))],
                [sg.Text("Reward", size=(20, 3), key="-REWARD-", justification='center')],
                [
                    sg.Button("Start", key="START", size=(10, 2)), 
                    sg.Button("Terminate", key="TERMINATE", size=(10, 2)),
                ]
            ],
            element_justification='center'),
        ],
    ]

    window = sg.Window("Results", layout, margins=(25, 25))

    timer_running = True
    
    # Create an event loop
    while True:
        event, values = window.read(timeout = 10)

        if event == "TERMINATE" or event == sg.WIN_CLOSED:
            break
        elif event == "START":
            timer_running = not timer_running
        if timer_running:
            window['-REWARD-'].update(time.strftime("%H:%M:%S"))
            
    window.close()